In [1]:
import pandas as pd
import faiss
import os
import json

import tensorflow_hub as hub

from pprint import pprint

import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

NOT AS ACCURATE AS THE NON-CHUNKED VERSION. DO NOT USE!

In [2]:
legal_docs = pd.read_csv("../data/legal_text_classification.csv")
index = faiss.read_index("chunked_court_text.index")

with open('chunk-doc-map.json', 'r') as fp:
    chunk_to_doc_map = json.load(fp)

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [4]:
query1 = "claims related to shoe companies"
#query1 = "claims involving large sums of money"

# Encode the query into an embedding
query_embedding = embed([query1]).numpy()

# Search for similar embeddings in the index
D, I = index.search(query_embedding, k=5)  # Retrieve the top 5 similar documents

# Print the indices and distances of the similar documents
print("Top 5 similar documents:")
indices = []
for chunk_idx, d in zip(I[0], D[0]):
    doc_idx = chunk_to_doc_map[str(chunk_idx)]
    print(f"Index: {doc_idx}, Similarity Score: {1 - d}")
    indices.append(doc_idx)

Top 5 similar documents:
Index: 5892, Similarity Score: -0.17656362056732178
Index: 22065, Similarity Score: -0.27195632457733154
Index: 4448, Similarity Score: -0.2751309871673584
Index: 9587, Similarity Score: -0.2851524353027344
Index: 23104, Similarity Score: -0.33100032806396484


In [8]:
idx = 9587

pprint (legal_docs.iloc[idx]["case_title"])
pprint (legal_docs.iloc[idx]["case_text"])

'Equuscorp Pty Ltd v Perpetual Trustees WA Ltd (1997) 25 ACSR 675'
('G &amp; J Gears conceded that it had not satisfied the statutory demand '
 'within the period for compliance allowed by the registrar, but it contended '
 'that the court could extend the time for compliance because it had made an '
 'application for an extension before the time fixed by the registrar had '
 'passed. In support of this contention, it referred to Equuscorp Pty Ltd v '
 'Perpetual Trustees WA Ltd (1997) 25 ACSR 675 (" Equuscorp ") at 701 per '
 'French, Kiefel and Sundberg JJ. G &amp; J Gears contended that the court '
 'should exercise the discretion it had in its favour, amongst other reasons, '
 'because the review would otherwise be nugatory. 36 Further, G &amp; J Gears '
 'also relied on s 459F(2)(a)(ii) of the Corporations Act in support of its '
 'submission that the Court could and should make an order extending the time '
 'for compliance with the demand, notwithstanding that the time allowed u

In [10]:
def answer_question(question, text, num_sentences=5, english_level=9):
  response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    temperature = 0,
    messages = [{"role":"system", "content": f"Follow these instructions when writing the answer:\
      \n1. Different texts in the input are separated by ***"
      },
      {"role": "user", "content":f"Answer the following question about the text:{question}. text:{text}"}]
    )

  return response['choices'][0]['message']['content']


def summarize_text(text, num_sentences=5, english_level=9):
  response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    temperature = 0,
    messages = [{"role":"system", "content": f"Follow these instructions when writing the summary:\
      \n1. Write a clear and concise summary consisting of {num_sentences} sentences \
      \n2. The summary's english level matches that of a person with {english_level} years of education \
      \n3. The summary should consist of an explanation of what the case is about, who's involved and the outcome"
      },
      {"role": "user", "content":f"Write a summary of the following text:{text}"}]
    )

  return response['choices'][0]['message']['content']

In [13]:
query_docs_list = legal_docs.iloc[indices]["case_text"].tolist()
texts = []
for query_doc in query_docs_list:
  texts.append(query_doc)

text_all = "***".join(texts)


In [16]:
pprint (answer_question("What's common in all of these cases?", text_all))

('***In all of these cases, the issue of knowledge is important in determining '
 'liability for copyright infringement. The courts consider the knowledge, '
 'capacity, and circumstances of the defendant when determining whether they '
 'had the requisite knowledge of the infringement. In the case of Raben '
 'Footwear Pty Ltd v Polygram Records Inc, the court found that knowledge '
 "could not be inferred from the defendant's pattern of trade. In Deckers "
 'Outdoor Corporation Inc v Farley, the court made declarations and ordered '
 'the defendant to deliver up counterfeit footwear and pay damages for '
 "infringing Deckers' intellectual property rights. In World Brands Management "
 'Pty Ltd v Cube Footwear Pty Ltd, the court considered the common process of '
 'purchasing footwear and the expectations of retail consumers. In Lockwood '
 'Security Products Pty Ltd v Doric Products Pty Ltd, the court discussed fair '
 'basis and the comparison between the claims made in a patent and

In [14]:
pprint (answer_question("What's common to all the companies mentioned in this text?", text_all))


('The commonality among all the companies mentioned in this text is that they '
 'are involved in legal proceedings or disputes.')


In [15]:
pprint (answer_question("What's the common industry for all the companies in these texts?", text_all))


('The common industry for all the companies in these texts is the legal '
 'industry.')


In [11]:
text = legal_docs.iloc[idx]["case_text"]
pprint (answer_question("What is this document about?", text))


('This document is about a case involving G & J Gears and their failure to '
 'comply with a statutory demand within the allowed time period. G & J Gears '
 'argues that the court should extend the time for compliance because they had '
 "made an application for an extension before the registrar's deadline. They "
 'also rely on a section of the Corporations Act to support their argument. G '
 '& J Gears further contends that the statutory demand should be set aside due '
 'to offsetting claims they have against Brobo, including trademark '
 'infringement, breach of contract, and unlawful conversion of designs. The '
 'document also mentions that G & J Gears had the opportunity to apply for a '
 'further extension of time, but they did not do so. Once the time for '
 'compliance has passed, the court no longer has the power to extend it.')


In [17]:
pprint (answer_question("Who is the plaintiff in this case and what are they looking for?", text))


('The plaintiff in this case is G & J Gears. They are looking for the '
 'statutory demand to be set aside because they believe they have offsetting '
 'claims against Brobo. These claims include a trademark infringement claim, a '
 'claim of unlawful inducement to breach a distribution agreement, a breach of '
 'contract claim, and a claim of unlawful conversion of designs, patterns, and '
 'tools. G & J Gears argues that these offsetting claims should be considered '
 'and that the court should exercise its discretion in their favor.')


In [26]:
pprint (summarize_text(text, english_level=5))


('The case mentioned is World Brands Management Pty Ltd v Cube Footwear Pty '
 'Ltd [2004] FCA 769. The judge, Heerey J, refers to a previous case, Dr '
 'Martens Australia Pty Ltd v Figgins Holdings Pty Ltd (1999) 44 IPR 281, but '
 'does not agree that it establishes a rule of law for the retail footwear '
 'trade. However, Heerey J finds value in the observations made in the '
 'previous case, as they align with personal experiences of how shoes are '
 'typically purchased. Shoes are not usually bought impulsively, but rather, '
 'consumers compare, try on, and think over their options. Shoes are expected '
 'to last for a few years, and if not chosen well, they can cause discomfort '
 'and regret. The outcome of the case is not mentioned in the summary.')


In [27]:
pprint (answer_question("What is the meaning of life?", text_all))


'The meaning of life is not discussed in the given texts.'
